Importing Necessary Packages

In [9]:
import tarfile, urllib, os
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline
import matplotlib as mpl
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

Fetching Dataset

In [10]:
url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz"
path = os.path.join("datasets", "housing") 
filepath = os.path.join(path, "housing.tgz")
os.makedirs(path)
urllib.request.urlretrieve(url, filepath)
housing_tgz = tarfile.open(filepath)
housing_tgz.extractall(path=path)
housing_tgz.close()

Load Dataset

In [11]:
df = pd.read_csv("datasets\housing\housing.csv")
df.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
